In [1]:
import requests
import pandas as pd
import json

In [2]:
def load_api_key(key_path: str):
    with open(key_path, mode='r') as file:
        API_KEY = file.read()
    return API_KEY


def get_sports_data(api_key: str, return_data: bool = False, return_headers: bool = False):

    sports_response = requests.get('https://api.the-odds-api.com/v4/sports', params={
        'api_key': api_key
    })

    if sports_response.status_code != 200:
        print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')
    else:
        # write sports data to database in "in_season_sports" table
        df = pd.DataFrame(sports_response.json())
        # write headers info to database in "in_season_sports_headers" table
        headers = sports_response.headers
        headers_df = pd.DataFrame(data=dict(headers), index=[0])
    
    if return_data:
        return df
    if return_headers:
        return headers_df
    
    return df, headers_df


def get_odds_data(api_key: str, sport: str, regions: str, markets: str, odds_format: str, date_format: str):

    odds_response = requests.get(f'https://api.the-odds-api.com/v4/sports/{sport}/odds', params={
        'api_key': api_key,
        'regions': regions,
        'markets': markets,
        'oddsFormat': odds_format,
        'dateFormat': date_format,
    })

    with open(r'/home/carls/projects/odds_site/odds_site_db/epl_odds_response.json', mode='w') as json_file:
        odds_json = json.dump(odds_response.json(), json_file)

    extra_meta = [['bookmakers', 'key'], ['bookmakers', 'title'], ['bookmakers', 'last_update'], ['bookmakers', 'markets', 'last_update'], ['bookmakers', 'markets', 'key']]
    meta = pd.json_normalize(data=odds_response.json()).columns.to_list()[:-1] + extra_meta

    odds_df = pd.json_normalize(
        data=odds_response.json(),
        meta=meta,
        record_path=['bookmakers', 'markets', 'outcomes'],
    )

    odds_headers_df = pd.DataFrame(dict(odds_response.headers), index=[0])

    return odds_df, odds_headers_df, odds_response

In [3]:
DB_PATH = r'/home/carls/projects/odds_site/odds_site_db/'
BASE_URL = 'https://api.the-odds-api.com/v4/sports'

APIKEY_PATH = r'/home/carls/projects/odds_site/api_key/odds_api_key.txt'
# API_KEY = load_api_key(key_path = APIKEY_PATH)

In [4]:
SPORT = 'soccer_epl'
REGIONS = 'us'
MARKETS = 'h2h'
ODDS_FORMAT = 'decimal'
DATE_FORMAT = 'iso'

# data, headers, res = get_odds_data(
#     api_key=API_KEY, 
#     sport=SPORT,
#     regions=REGIONS,
#     markets=MARKETS,
#     odds_format=ODDS_FORMAT,
#     date_format=DATE_FORMAT
# )

In [5]:
# data

In [6]:
# headers

In [7]:
# res.json()